In [1]:
# Specify the folder path
folder_path = 'imgs\\100_0824\\'
host='http://192.168.86.68:11434'
generation_model = 'llava:34b'
validation_model = 'llava:13b' #'llava:7b' 'moondream'

In [2]:
from PIL import Image
import pprint
try:
    import exifread
except ModuleNotFoundError:
    ! pip install exifread
    import exifread
try:
    import ollama
except ModuleNotFoundError:
     ! pip install ollama
from ollama import Client
client = Client(host=host)

import time

try:
    import piexif
except ModuleNotFoundError:
    ! pip install piexif
    import piexif
from PIL import Image

import os
import glob



def add_exif_data(image_path, title=None):
    image = Image.open(image_path)
    
    # Load exif data
    try:
        originalexif_dict = piexif.load(image.info["exif"])
        exif_dict = piexif.load(image.info["exif"])
    except KeyError:
        print("No EXIF data in this file.")
        return

    if title is not None:
        exif_dict['0th'][piexif.ImageIFD.ImageDescription] = title 
        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, image_path)


def update_queries(img, Queries, client, generation_model, validation_model):
    """
    This function takes in a dictionary of queries and their corresponding results.
    
    Parameters:
    img: The image for which descriptions are being generated.
    Queries (dict): A dictionary where each key is a query name and each value is a query object.
    client: The client instance that interacts with the LLaMA API.
    generation_model: The model used for generating descriptions from images.
    validation_model: The model used to validate generated descriptions.

    Returns:
    A dictionary where each key is a query name and each value is the generated description for that query.
    """

    # Initialize an empty dictionary to store the results
    results_dict = {}
  
    for Query_name, Query in Queries.items():
        for i in range(5):
            # Generate description for the image
            description = client.generate(generation_model, Query['generation'], images=[img])
            
            # Validate the generated description with a validation model
            descriptioncheck = client.generate(validation_model, Query['validation'] + description['response'], images=[img])

            if len(description['response']) < Query['limit']:
                for j in range(5):
                    if descriptioncheck['response'].lstrip().lower().startswith('yes') or descriptioncheck['response'].lstrip().lower().startswith('no'):
                        break
                if descriptioncheck['response'].lstrip().lower().startswith('yes'):
                    break

 #       if i > 0:
 #           print('\033[1m\033[91m' + Query_name.capitalize() + " " +str(i) + ":\033[0m\033[0m " + description['response'] + "\n\n\n")
 #       elif len(description['response']) > Query['limit']:
 #           print('\033[1m\033[91m' + Query_name.capitalize() + str(i + " ERROR Getting Small enough Result - "  + len(description['response']) + " ") + ":\033[0m\033[0m " + description['response'] + "\n\n\n")
 #       else:
 #           print("\033[1m" + Query_name.capitalize() + ":\033[0m "  + description['response'] + "\n\n\n")

        # Store the result in the results dictionary
        results_dict[Query_name] = description['response']

    return results_dict


def get_images_with_empty_ImageDescription(folder_path):
  """
  This function finds all image files in a folder and returns a list of filenames
  where the EXIF ImageDescription is empty.

  Args:
      folder_path (str): The path to the folder containing images.

  Returns:
      list: A list of filenames of images with empty EXIF titles.
  """
  images_with_empty_title = []
  for filename in os.listdir(folder_path):
    # Check if it's a file (not a directory) and has an image extension
    if os.path.isfile(os.path.join(folder_path, filename)) and filename.lower().endswith((".jpg", ".jpeg")):
      try:
        print(f"Acessing image or EXIF data for: {filename}")
        image = Image.open(os.path.join(folder_path, filename))
        exif_dict = piexif.load(image.info["exif"])
        # Check if title is missing (assuming '0th' IFD) or an empty string
        if exif_dict.get("0th", {}).get(piexif.ImageIFD.ImageDescription, "") == "":
          images_with_empty_title.append(filename)
        else:
            print(exif_dict.get("0th", {}).get(piexif.ImageIFD.ImageDescription, ""))
      except (OSError, KeyError):
        # Handle potential errors like file not found or missing EXIF data
        print(f"Error accessing image or EXIF data for: {filename}")

  return images_with_empty_title


In [3]:
Queries = {
    "title": {
        "generation": "Title this image in less than 12 words",
        "validation": "Answer only with Yes or No. Is the following a good title for this image?",
        "limit": 150
    },
}


In [ ]:


# List all JPEG files in the folder
jpeg_files = glob.glob(os.path.join(folder_path, '*.JPG'))
#jpeg_files = get_images_with_empty_ImageDescription(folder_path)

#print(jpeg_files)

batchstart = time.time()

# Loop through each JPEG file and update the queries
for img in jpeg_files:
    #print(img)
    # Pass the image file into the update_queries function
    imagestart = time.time()
    results = update_queries(img, Queries, client, generation_model, validation_model)
    imageend = time.time()
    add_exif_data(img, results['title'] )
    print("Execution time of generating exif data for " + img + " is " + str(round((imageend - imagestart), 0)) + " seconds. " + results['title'])

batchend = time.time()
print("\n\nExecution time of generating exif data for " + str(len(jpeg_files)) + " is " + str(round((batchend - batchstart) / 60, 2)) + " minutes. ")


Execution time of generating exif data for imgs\100_0824\PANO0001.JPG is 21.0 seconds. An aerial view of a field.
Execution time of generating exif data for imgs\100_0824\PANO0002.JPG is 11.0 seconds. Abandoned Property, Overgrown Garden
Execution time of generating exif data for imgs\100_0824\PANO0003.JPG is 11.0 seconds. Aerial view of rural landscape.
Execution time of generating exif data for imgs\100_0824\PANO0004.JPG is 21.0 seconds. A serene landscape with bare trees.
Execution time of generating exif data for imgs\100_0824\PANO0005.JPG is 28.0 seconds. Winding Path Through Fallen Trees
Execution time of generating exif data for imgs\100_0824\PANO0006.JPG is 9.0 seconds. Path in forest with water on sides
Execution time of generating exif data for imgs\100_0824\PANO0007.JPG is 11.0 seconds. "Hills and Clouds, A View from Above"
Execution time of generating exif data for imgs\100_0824\PANO0008.JPG is 18.0 seconds. "Pathway between grassy areas."
Execution time of generating exif 